In [1]:
# Import libraries
from pathlib import Path
import time
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import nglview as nv
import openbabel
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from plip.structure.preparation import PDBComplex
from plip.exchange.report import BindingSiteReport

from opencadd.structure.core import Structure

In [2]:
import csv

In [3]:
# Absolute path
HERE = Path(_dh[-1])
DATA = HERE / "pdb"

In [4]:
HERE

PosixPath('/home/mxc/jupyterWorkspace/PLIP/palm2')

In [5]:
def retrieve_plip_interactions(pdb_file):
    """
    Retrieves the interactions from PLIP.

    Parameters
    ----------
    pdb_file :
        The PDB file of the complex.

    Returns
    -------
    dict :
        A dictionary of the binding sites and the interactions.
    """
    protlig = PDBComplex()
    protlig.load_pdb(pdb_file)  # load the pdb file
    for ligand in protlig.ligands:
        protlig.characterize_complex(ligand)  # find ligands and analyze interactions
    sites = {}
    # loop over binding sites
    for key, site in sorted(protlig.interaction_sets.items()):
        binding_site = BindingSiteReport(site)  # collect data about interactions
        # tuples of *_features and *_info will be converted to pandas DataFrame
        keys = (
            "hydrophobic",
            "hbond",
            "pistacking",
            "saltbridge",
            "pication",
            "halogen",
        )
        # interactions is a dictionary which contains relevant information for each
        # of the possible interactions: hydrophobic, hbond, etc. in the considered
        # binding site. Each interaction contains a list with
        # 1. the features of that interaction, e.g. for hydrophobic:
        # ('RESNR', 'RESTYPE', ..., 'LIGCOO', 'PROTCOO')
        # 2. information for each of these features, e.g. for hydrophobic
        # (residue nb, residue type,..., ligand atom 3D coord., protein atom 3D coord.)
        interactions = {
            k: [getattr(binding_site, k + "_features")] + getattr(binding_site, k + "_info")
            for k in keys
        }
        sites[key] = interactions
    return sites

In [6]:
def create_df_from_binding_site(selected_site_interactions, interaction_type="hbond"):
    """
    Creates a data frame from a binding site and interaction type.

    Parameters
    ----------
    selected_site_interactions : dict
        Precaluclated interactions from PLIP for the selected site
    interaction_type : str
        The interaction type of interest (default set to hydrogen bond).

    Returns
    -------
    pd.DataFrame :
        DataFrame with information retrieved from PLIP.
    """

    # check if interaction type is valid:
    valid_types = [
        "hydrophobic",
        "hbond",
        "pistacking",
        "saltbridge",
        "pication",
        "halogen",
    ]

    if interaction_type not in valid_types:
        print("!!! Wrong interaction type specified. Hbond is chosen by default!!!\n")
        interaction_type = "hbond"

    df = pd.DataFrame.from_records(
        # data is stored AFTER the column names
        selected_site_interactions[interaction_type][1:],
        # column names are always the first element
        columns=selected_site_interactions[interaction_type][0],
    )
    return df

In [11]:
protein_data = pd.read_csv(HERE/"palm2.csv").reset_index(drop=True)
protein_data

PDBID
0   4KHM
1   3FQL
2   4KB7
3   4KBI
4   4KE5
5   3FQK
6   5TRI
7   4KAI
8   4KHR
9   5TRH
10  5W2E

In [8]:
index_of_selected_site = 0

In [14]:
for i in range(len(protein_data)):
    pdb_id = protein_data['PDBID'][i]
    pdb_file = Structure.from_pdbid(pdb_id)
    # Download it to file for later use
    pdb_file.write(DATA / f"{pdb_id}.pdb")
    interactions_by_site = retrieve_plip_interactions(f"{DATA}/{pdb_id}.pdb")
    #interactions.append(str(pdb_id+interactions_by_site.keys()))
   # interactions.to_csv('interactions.txt', mode='a', sep='\t', index=False)
    selected_site = list(interactions_by_site.keys())[index_of_selected_site]
    #df = pd.DataFrame(zip(*(selected_site.replace(':',',').split(','))))
    #df.to_csv('hydrophobic.csv', mode='a', index=False, header=True)
    with open(HERE / "hydrophobic.csv",mode='a',newline='') as cf:
        print(f"$Number of binding sites detected in {pdb_id} : "
              f"{len(interactions_by_site)}\t"
              f"with {interactions_by_site.keys()}\n"
              f"$selected {selected_site}",file=cf)
        cf.close()
    create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="hydrophobic").to_csv('hydrophobic.csv', mode='a', index=False, header=False)

In [ ]:
 #df.to_csv('hbond.csv', mode='a', index=False, header=True)
    with open(HERE / "hbond.csv",mode='a',newline='') as cf:
        print(f"$Number of binding sites detected in {pdb_id} : "
              f"{len(interactions_by_site)}\t"
              f"with {interactions_by_site.keys()}\n"
              f"$selected {selected_site}",file=cf)
        cf.close()
    create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="hbond").to_csv('hbond.csv', mode='a', index=False, header=False)
    #df.to_csv('pistacking.csv', mode='a', index=False, header=True)
    with open(HERE / "pistacking.csv",mode='a',newline='') as cf:
        print(f"$Number of binding sites detected in {pdb_id} : "
              f"{len(interactions_by_site)}\t"
              f"with {interactions_by_site.keys()}\n"
              f"$selected {selected_site}",file=cf)
        cf.close()
    create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="pistacking").to_csv('pistacking.csv', mode='a', index=False, header=False)
    with open(HERE / "saltbridge.csv",mode='a',newline='') as cf:
        print(f"$Number of binding sites detected in {pdb_id} : "
              f"{len(interactions_by_site)}\t"
              f"with {interactions_by_site.keys()}\n"
              f"$selected {selected_site}",file=cf)
        cf.close()
    create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="saltbridge").to_csv('saltbridge.csv', mode='a', index=False, header=False)
    #df.to_csv('hbond.csv', mode='a', index=False, header=True)
    with open(HERE / "pication.csv",mode='a',newline='') as cf:
        print(f"$Number of binding sites detected in {pdb_id} : "
              f"{len(interactions_by_site)}\t"
              f"with {interactions_by_site.keys()}\n"
              f"$selected {selected_site}",file=cf)
        cf.close()
    create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="pication").to_csv('pication.csv', mode='a', index=False, header=False)
    #df.to_csv('pistacking.csv', mode='a', index=False, header=True)
    with open(HERE / "halogen.csv",mode='a',newline='') as cf:
        print(f"$Number of binding sites detected in {pdb_id} : "
              f"{len(interactions_by_site)}\t"
              f"with {interactions_by_site.keys()}\n"
              f"$selected {selected_site}",file=cf)
        cf.close()
    create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="halogen").to_csv('halogen.csv', mode='a', index=False, header=False)
#df = pd.DataFrame(zip(*(selected_site.replace(':',',').split(','))))
#df = pd.DataFrame(selected_site.replace(':',',').split(','))